In [1]:
import numpy as np
import pandas as pd
import xgboost
from sklearn.datasets import make_classification, make_moons, make_circles
from sklearn.preprocessing import StandardScaler

In [2]:
%matplotlib inline


import seaborn as sns

import matplotlib.pyplot as plt
#!pip install plotly
import plotly
import plotly.graph_objs as go

from matplotlib.colors import ListedColormap

How big dataset?

In [3]:
n_rows = int(1e4) #10M rows
n_columns = int(100)
n_classes = 2



In [4]:
df = make_classification(n_samples=n_rows, 
                         n_features=n_columns,
                         n_classes=n_classes, 
                         n_informative=int(n_columns/5),
                         n_clusters_per_class = 1,
                         class_sep = 1.0,
                         flip_y=0.2,
                         random_state=1)
pd_df = pd.DataFrame(df[0],columns=['x'+str(i) for i in range(1,n_columns+1)])
pd_df['y'] = df[1]
print(pd_df.head())

         x1        x2        x3        x4        x5        x6        x7  \
0  0.493722  1.003377  0.567122 -2.078380  0.486073  2.728119  1.390538   
1  0.155916  1.160258 -1.153349 -0.084261 -0.415886  1.923980 -0.667563   
2 -1.821358  1.895874  1.301383 -0.706522  0.264518 -2.222414 -1.329729   
3 -0.270944 -0.499991  2.259192  0.600572 -0.613155 -1.472841 -0.404570   
4 -0.665526 -0.395929  0.724794 -0.764808 -0.986461  0.168381 -0.932520   

         x8        x9       x10  ...       x92       x93       x94       x95  \
0 -0.545780  1.657025 -0.159087  ... -0.127669  0.251150 -0.759959 -0.043082   
1  2.487999 -0.543920  1.702723  ...  0.701373  2.125454 -1.762217  4.872770   
2  0.965604  1.174375 -1.380014  ...  0.122855  0.813238  2.023493  2.237147   
3 -0.257546 -1.962031 -0.146880  ... -1.997139  0.054975  1.323091 -2.431615   
4 -0.425686  0.369523  0.125641  ... -0.171411 -0.677577 -0.451854 -4.012032   

        x96       x97       x98       x99      x100  y  
0 -0.288428

###  1. Train / Test split 

In [5]:
X, y = df[0],df[1]

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [7]:
len(X_train), len(X_test)


(8000, 2000)

In [8]:
%%time
pd.DataFrame(data = X_train).to_csv('X_train.csv')
pd.DataFrame(data = X_test).to_csv('X_test.csv')
pd.DataFrame(data = y_train).to_csv('y_train.csv')
pd.DataFrame(data = y_test).to_csv('y_test.csv')

CPU times: user 1.77 s, sys: 165 ms, total: 1.93 s
Wall time: 1.77 s
